In [1]:
import pandas as pd
import plotly_express as px

In [2]:
# Load the HTML table into a DataFrame
trades = pd.read_html('./Statement.htm')
df = trades[0].copy()  

header_row_index = 2
df.columns = df.iloc[header_row_index]

# Remove the header row and rows above it
df = df[header_row_index + 1:]

# Drop unwanted columns without using inplace=True
df = df.drop(['Ticket', 'Taxes'], axis=1)

# Rename columns
df.columns = ['Open Date', 'Type', 'Size', 'Asset', 'Open Price', 'Stop Loss', 'Take Profit', 'Close Date', 'Close Price', 'Commissions', 'Swap', 'PnL']

# Filter rows based on 'Type'
valid_trade_types = ['buy', 'sell', 'buy limit', 'sell limit', 'buy stop', 'sell stop']
df = df[df['Type'].str.lower().isin(valid_trade_types)]

# Filter out rows where 'PnL' equals 'cancelled'
df = df[df['PnL'].str.lower() != 'cancelled']

# Convert specific columns to float
cols_to_float = ['Size', 'Open Price', 'Stop Loss', 'Take Profit', 'Close Price', 'Commissions', 'Swap', 'PnL']
for col in cols_to_float:
    df[col] = df[col].astype(str).str.replace(' ', '').astype(float)

df['Open Date'] = pd.to_datetime(df['Open Date'], format='%Y.%m.%d %H:%M:%S')
df['Close Date'] = pd.to_datetime(df['Close Date'], format='%Y.%m.%d %H:%M:%S')
df = df.sort_values(by='Close Date', ascending=True)
df['Rolling PnL'] = df['PnL'].cumsum()



In [3]:
df['Close Date'] = pd.to_datetime(df['Close Date']).dt.date
daily_pnl_df = df.groupby('Close Date')['PnL'].sum().reset_index()
daily_pnl_df['Rolling PnL'] = daily_pnl_df['PnL'].cumsum()


In [14]:
def calculate_real_drawdown(daily_pnl_df):
    new_highs = []
    drawdowns = []
    drawdown_dates = []
    drawdown_durations = []
    current_high = daily_pnl_df['Rolling PnL'].iloc[0]
    lowest_rolling = current_high
    date_of_lowest_rolling = daily_pnl_df['Close Date'].iloc[0]

    for index, row in daily_pnl_df.iterrows():
        if row['Rolling PnL'] > current_high:
            current_high = row['Rolling PnL']
            new_highs.append((row['Close Date'], current_high))

            drawdown_amount = current_high - lowest_rolling
            drawdowns.append(drawdown_amount)
            
            start_date = date_of_lowest_rolling
            end_date = row['Close Date']
            drawdown_dates.append((start_date, end_date))

            duration = (end_date - start_date).days
            drawdown_durations.append(duration)

            lowest_rolling = current_high
            date_of_lowest_rolling = row['Close Date']
        else:
            if row['Rolling PnL'] < lowest_rolling:
                lowest_rolling = row['Rolling PnL']
                date_of_lowest_rolling = row['Close Date']

    return new_highs, drawdowns, drawdown_dates, drawdown_durations

def calculate_largest_drawdown(daily_pnl_df):
    _, drawdowns, drawdown_dates, _ = calculate_real_drawdown(daily_pnl_df)

    if drawdowns:
        largest_drawdown = max(drawdowns)
        largest_drawdown_index = drawdowns.index(largest_drawdown)
        start_date, end_date = drawdown_dates[largest_drawdown_index]

        duration = (end_date - start_date).days
        return largest_drawdown, start_date, end_date, duration
    else:
        return None, None, None, None

def average_drawdown_duration(daily_pnl_df):
    _, _, _, drawdown_durations = calculate_real_drawdown(daily_pnl_df)

    if drawdown_durations:
        average_duration = sum(drawdown_durations) / len(drawdown_durations)
        return average_duration
    else:
        return None


In [15]:
largest_drawdown, start_date, end_date, duration = calculate_largest_drawdown(daily_pnl_df)
avg_duration = average_drawdown_duration(daily_pnl_df)

In [23]:
avg_duration

2.8

In [19]:
daily_pnl_df.head(50)

,Close Date,PnL,Rolling PnL
0,2023-09-08,-2003.74,-2003.74
1,2023-09-12,-274.58,-2278.32
2,2023-09-13,-1417.88,-3696.20
3,2023-09-14,2389.08,-1307.12
4,2023-09-15,1714.92,407.80
5,2023-09-18,-524.48,-116.68
6,2023-09-19,-1339.11,-1455.79
7,2023-09-20,2243.77,787.98
8,2023-09-22,2669.64,3457.62
9,2023-09-25,-899.66,2557.96
